# Agentic summarization example

In [1]:
import os
from IPython.display import display, HTML, Markdown
from dotenv import load_dotenv
from pathlib import Path
import re
import base64
import tiktoken
import google.generativeai as genai
from openai import OpenAI

In [2]:
load_dotenv()

True

In [3]:
openai_client = OpenAI()
genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

In [4]:
THEME = "neurological/neurogenic heterotopic ossification"
PDF_PATH = "articles"

In [5]:
supervisor_prompt = f"""
You are a medical research assistant. The first input you will receive will be a medical disease, condition or/and a more generic topic on a class of medical disorders.
Your task will be to generate a detailed, professional clinical guideline and best practice recommendation for diagnosing and treating patients with the medical disease or condition in a Hungarian rehabilitation center, tailored towards guiding PM&R specialists responsible for inpatient care at a highly professional neurorehabilitation department. The guideline is not meant to be a policy recommendation, advising policy makers how to organize the medical processes, instead it should focus on the individual physicians work and the medical professional and scientific aspect, while also providing clear clinical algorithms on dealing with a specific medical disease or condition. /

The guideline should be very detailed and having all the necessary information for an attending PM&R physician how to manage patients with the mentioned conditions. It should be extremely specific, with detailed instructions, comprehensive process descriptions detailing key decision points on how to deal with patients suffering from the specified disease, condition or disorder.

You will have access to a number of relevant scientific papers, reviews, existing guidelines and an LLM agent that is responsible for extracting important information for you from the PDF documents based on your guidance on what parts and what kind of information it should focus on, and will return to you all the extracted details for generating the guidelines and augmenting your knowledge on the topic with the latest research and up-to-date existing knowledge.

First provide brief, concise instructions for your LLM agent for processing the external documentations, assisting it to focus on the relevant details. Put the instructions within <instructions></instructions> tags. You do not need to write the guideline yet, instead wait for the results from your LLM agents.

The relevant medical disease, condition, disorder for which you need to propose the guideline: {THEME}
"""

In [6]:
extractor_prompt = """
You are a medical research assistant. You will receive some scientific article, review or medical recommendation in PDF format and the detailed instruction on what kind of relevant medical information the research supervisor is interested in for the project. Process the document according to the instructions, while providing longer explanation on novel ideas and knowledge not well known previously within the context of the research.

Do not forget to add the title of the processed article in the header part.

# Detailed instructions:

{}
"""

In [7]:
writer_prompt = """
You are a medical research assistant. The first input you will receive will be a medical disease, condition or/and a more generic topic on a class of medical disorders.
Your task will be to generate a detailed, professional clinical guideline and best practice recommendation for diagnosing and treating patients with the medical disease or condition in a Hungarian rehabilitation center, tailored towards guiding PM&R specialists responsible for inpatient care at a highly professional neurorehabilitation department. The guideline is not meant to be a policy recommendation, advising policy makers how to organize the medical processes, instead it should focus on the individual physicians work and the medical professional and scientific aspect, while also providing clear clinical algorithms on dealing with a specific medical disease or condition. /

The guideline should be very detailed and having all the necessary information for an attending PM&R physician how to manage patients with the mentioned conditions. It should be extremely specific, with detailed instructions, comprehensive process descriptions detailing key decision points on how to deal with patients suffering from the specified disease, condition or disorder.

You will receive a number of extracted notes and documents below within <documents></documents> tags from fresh research papers, reviews and existing guidelines, which will provide up-to-date research for you to extend your knowledge. Carefully review these documents, and use all additional facts, novel findings and recommendations to extend the final guidelines will all the best available knowledge to generate a long writeup in markdown format, which provides all the latest facts, research and details extracted from the documents as well as your latest and best knowledge available to you to propose a comprehensive guideline.

The relevant medical disease, condition, disorder for which you need to propose the guideline: {}

# External knowledge
<documents>
{}
</documents>
"""

### Supervisor agent

In [8]:
response = openai_client.responses.create(
  model="o1",
  input=supervisor_prompt
)

In [9]:
pattern = r"<instructions>(.*?)</instructions>"
extracted_instructions = re.search(pattern, response.output_text, re.DOTALL).group(1)

In [10]:
Markdown(extracted_instructions)


1. Focus on the pathophysiology, epidemiology, and risk factors of neurological (neurogenic) heterotopic ossification (HO) that appear in the documents.  
2. Extract information on early recognition and screening protocols, including clinical assessments, biomarkers, and imaging modalities for diagnosing neurogenic HO.  
3. Identify and summarize the recommended medical management strategies (e.g., NSAIDs, bisphosphonates, etc.) and their evidence-based rationale.  
4. Gather details on recommended rehabilitation interventions, such as physical therapy approaches, spasticity management, positioning, and splinting.  
5. Locate any surgical guidelines or indications for surgical intervention in neurogenic HO, including postoperative rehabilitation considerations, if available.  
6. Collect data on prognostic factors, functional outcomes, and long-term follow-up recommendations within inpatient neurorehabilitation settings.  
7. Extract any relevant clinical algorithms or step-by-step approaches outlined in the sources for diagnosing and managing neurogenic HO, including patient and family education.  
8. Re-check the documents for novel or emerging therapies, recent review articles, or updated guidelines on neurogenic HO.  
9. Prioritize any content that targets PM&R specialists working in a rehabilitation center in Hungary, focusing on practical, inpatient care considerations.  


### Extractor agents

In [11]:
model_extracting = genai.GenerativeModel('gemini-2.0-flash-exp')

In [12]:
documents = []

for f in Path(PDF_PATH).glob('**/*'):

    with open(f, "rb") as doc_file:
        doc_data = base64.standard_b64encode(doc_file.read()).decode("utf-8")

    print(f)
    response = model_extracting.generate_content([{'mime_type': 'application/pdf', 'data': doc_data}, extractor_prompt.format(extracted_instructions)])
    documents.append(f'<document>\n{response.text}\n</document>')

articles\06SAKELLARIOU.pdf
articles\1-s2.0-S1877056817303274-main.pdf
articles\1-s2.0-S1877065716300306-main.pdf
articles\1-s2.0-S2214031X17300396-main.pdf
articles\121-1591873437.pdf
articles\1440-1681_13025 -- b7f720263ac5067ece2bdbf1d803cc3a.pdf
articles\2015-12-32-neuro-gil.pdf
articles\20210527-25446-1j9m9lr.pdf
articles\20240724-319105-r311o6.pdf
articles\20241031-374610-dxij7y.pdf
articles\3101309.pdf
articles\569165-4913911-1-PB.pdf
articles\62D03BA931D2E26A161F39185BFABE24.pdf
articles\arm-40-1135.pdf
articles\a_23_year_old_male_patient_with_history_of_severe.17.pdf
articles\bonejointres-02-51.pdf
articles\bot_0000000000000666 -- d0f459fbb296677ae0f6673b41671d30.pdf
articles\D7caAsWu.pdf
articles\ERABI-Module-11-Heterotopic-Ossification-V14.pdf
articles\fncel-16-842090.pdf
articles\gangloff-et-al-1993-neurogenic-heterotopic-ossification.pdf
articles\Guidelines-SCI-Heterotopic-Ossification-2024.pdf
articles\jbm4_10172 -- fe977f7ad8aae09f6f13bc16e5966701.pdf
articles\j_clineuro_

### Writer agent

In [13]:
joined_context = '\n'.join(documents)

In [16]:
# Checking the size of the context
enc = tiktoken.encoding_for_model("o1")
len(enc.encode(joined_context))

61049

In [17]:
model_writer = genai.GenerativeModel('gemini-2.0-flash-thinking-exp')

In [18]:
response = model_writer.generate_content(writer_prompt.format(THEME,joined_context))

In [19]:
Markdown(response.text)

# Clinical Guideline and Best Practice Recommendation for Neurological/Neurogenic Heterotopic Ossification in Hungarian Neurorehabilitation Centers

**For PM&R Specialists in Inpatient Neurorehabilitation Departments**

**1. Introduction: Understanding Neurogenic Heterotopic Ossification (NHO)**

Neurogenic Heterotopic Ossification (NHO) is a debilitating condition characterized by the abnormal formation of mature lamellar bone within extraskeletal soft tissues, typically surrounding joints. It is a frequent and significant complication following neurological injuries, particularly traumatic brain injury (TBI), spinal cord injury (SCI), stroke, and other central nervous system (CNS) insults. NHO can severely limit joint range of motion, cause pain, and impede functional recovery, posing a substantial challenge in neurorehabilitation settings.

This clinical guideline aims to provide a detailed, evidence-based framework for PM&R specialists in Hungarian rehabilitation centers to effectively diagnose, manage, and treat patients with NHO. It synthesizes the latest research findings and best practices to guide clinical decision-making in inpatient neurorehabilitation, focusing on the practical aspects of physician work and the medical-scientific dimensions of patient care. This guideline is intended to support individual physician practice and does not constitute a policy recommendation for healthcare system organization.

**2. Pathophysiology, Epidemiology, and Risk Factors of Neurogenic Heterotopic Ossification**

**2.1. Pathophysiology:**

NHO pathogenesis is complex and not fully elucidated, involving a combination of systemic and local factors initiated by CNS injury and often exacerbated by peripheral tissue trauma. The process generally follows these steps:

*   **Initiating Event:** CNS injury (TBI, SCI, stroke) and peripheral tissue trauma (muscle injury, fractures, microtrauma from spasticity and mobilization) trigger an inflammatory cascade.
*   **Inflammatory Phase:** Damage to the CNS disrupts the blood-brain barrier (BBB) and blood-spinal cord barrier (BSB), releasing inflammatory mediators (cytokines, growth factors, neuropeptides like Substance P, Oncostatin M, CGRP), and osteoinductive factors into systemic circulation. Local inflammation at the periarticular soft tissues is also crucial, involving macrophages, lymphocytes, and mast cells.
*   **Mesenchymal Stem Cell (MSC) Involvement:** Osteoprogenitor cells (OPCs), often originating from muscle tissue, are stimulated to proliferate and differentiate into chondrocytes and osteoblasts via complex signaling pathways, including Bone Morphogenetic Proteins (BMPs), Wnt/β-catenin, Hedgehog (HH), and mTOR pathways. Hypoxia within the injured tissue, often induced by immobilization and spasticity, plays a critical role by stabilizing HIF-1α, further promoting osteogenesis and angiogenesis.
*   **Endochondral Ossification:** NHO follows a process similar to endochondral ossification, involving mesenchymal condensation, chondrogenesis, cartilage hypertrophy, vascular invasion, and osteogenesis, ultimately leading to the formation of mature lamellar bone in ectopic locations.
*   **Neuro-immunological Factors:** Autonomic dysregulation, microvascular alterations, and altered neural signaling contribute to creating a permissive environment for osteogenesis.

**2.2. Epidemiology:**

*   **Incidence post-SCI:** 10-53% (clinically significant NHO: 20-30%, ankylosis: 3-8%)
*   **Incidence post-TBI:** 4-23% (clinically relevant joint immobility: ~10%)
*   **Incidence in Neurological Rehabilitation (non-traumatic):** ~4% (may be underestimated)
*   **Location:** Hip (most common, 70-97% in SCI), knee, elbow, shoulder. Often below the level of SCI or near large joints after TBI.
*   **Timing:** Typically develops within 1-6 months post-injury, with early signs as soon as 2-3 weeks.

**2.3. Risk Factors:**

*   **Neurological Injury Severity:** Severe TBI (GCS 3-8), complete SCI (AIS A), prolonged coma.
*   **Polytrauma:** Concomitant CNS and peripheral musculoskeletal injuries, especially fractures (femur, acetabular, elbow). Blast-related TBI is a particularly high-risk injury pattern.
*   **Immobilization:** Prolonged bed rest, limited joint mobility.
*   **Spasticity/Hypertonia:** Marked muscle spasticity is a significant independent risk factor, potentially causing muscle hypoxia and tears.
*   **Infections:** Urinary tract infections, pneumonia, pressure ulcers, systemic infections.
*   **Other Medical Complications:** Deep vein thrombosis (DVT), pressure ulcers.
*   **Patient Demographics:** Male sex, young age (20-30 years), older age (>30 years in some SCI studies), potentially Black race.
*   **Genetic Predisposition:** Possible associations with HLA-B18 and HLA-B27 (inconsistent findings), HLA B27 gene presence, genetic disorders like Fibrodysplasia Ossificans Progressiva (FOP).
*   **Prolonged Mechanical Ventilation:** Artificial ventilation, tracheostomy.
*   **Elevated Biomarkers:** Increased serum alkaline phosphatase (ALP), serum inflammatory markers (CRP, ESR, IL-6, TNF-α), urinary prostaglandin E2.
*   **Hormonal Factors:** Pituitary dysfunction post-TBI, altered growth hormone response, leptin levels.
*   **Medications:** Use of antidepressants, baclofen at admission to rehabilitation.
*   **Lifestyle Factors:** Tobacco smoking.
*   **Previous HO:** History of HO.

**3. Early Recognition and Screening Protocols**

**3.1. Clinical Assessment:**

*   **Be Vigilant:** In all patients with neurological injuries, especially those with risk factors, maintain a high index of suspicion for NHO.
*   **Regular Monitoring:** Regularly assess joints, particularly hips, knees, elbows, and shoulders, for early signs.
*   **Key Clinical Signs and Symptoms (appear 2-12 weeks post-injury, but can be earlier):**
    *   **Pain:** Localized joint or muscle pain, sometimes out of proportion to injury, may be less prominent in SCI patients with sensory deficits.
    *   **Decreased Range of Motion (ROM):** Progressive loss of joint mobility, stiffness, and a discrete endpoint with pain during passive movement.
    *   **Swelling:** Peri-articular swelling, firm mass, or edema.
    *   **Warmth and Erythema:** Localized warmth, redness, and inflammation around the joint.
    *   **Systemic Signs:** Low-grade fever, elevated inflammatory markers (CRP, ESR).
    *   **Increased Spasticity:** Worsening spasticity in the affected limb.
    *   **Functional Limitation:** Difficulty with transfers, mobility, and activities of daily living.
*   **Differential Diagnosis:** Rule out other conditions with similar presentations: deep venous thrombosis (DVT), septic arthritis, cellulitis, osteomyelitis, hematoma, fracture, tumor, musculoskeletal injuries, or inflammatory arthritis.

**3.2. Biomarker Screening (Consider as adjunct, not definitive):**

*   **Serum Alkaline Phosphatase (ALP):**  Measure serum ALP levels in high-risk patients. Elevated levels (2-3.5 times normal, peaking around 10 weeks post-injury) can support clinical suspicion, especially when combined with other signs. However, ALP is non-specific and can be elevated due to fracture healing or adolescent growth.
*   **Erythrocyte Sedimentation Rate (ESR) and C-Reactive Protein (CRP):**  Elevated ESR and CRP indicate inflammation and may be useful in the acute inflammatory phase, though also non-specific. Monitor CRP for correlation with inflammatory activity of HO.
*   **Urinary Prostaglandin E2 (PGE2):** Consider 24-hour urinary PGE2 excretion in patients with suspicious symptoms for early diagnosis. Elevated levels may indicate early HO formation.
*   **Creatine Kinase (CPK):** May be elevated in HO patients, suggesting muscle involvement.
*   **Emerging Biomarkers (Research context):** IL-6, IL-10, MCP-1, MMP-9, Substance P, Oncostatin M (OSM).

**3.3. Imaging Modalities (Step-wise Approach):**

*   **Step 1: Plain Radiography (X-ray):**
    *   Initial imaging modality, readily available and cost-effective.
    *   May be normal in early HO (first 3-6 weeks). Useful for detecting mature HO and monitoring progression.
*   **Step 2: Bedside Ultrasonography (US):**
    *   Consider as a first-line imaging tool, especially in resource-limited settings.
    *   Sensitive for detecting early soft tissue lesions, calcifications, and differentiating HO from other conditions (DVT, infection, hematoma).
    *   Bedside application, repeatable, inexpensive, no radiation.
    *   Power Doppler Ultrasound (PDUS) can assess vascularity.
*   **Step 3: Three-Phase 99m-Technetium Bone Scan:**
    *   Gold standard for early detection, highly sensitive (detects changes 2-6 weeks before radiographs).
    *   Useful for confirming early HO activity.
    *   Lower specificity, may not differentiate NHO from other inflammatory processes. Consider if clinical suspicion is high and US is inconclusive.
*   **Step 4: Computed Tomography (CT) Scan:**
    *   Essential for pre-surgical planning, provides detailed 3D visualization of HO extent and relationship to surrounding structures (vessels, nerves).
    *   CT angiography with contrast enhancement is the reference examination for pre-operative work-up.
*   **Step 5: Magnetic Resonance Imaging (MRI):**
    *   Useful for early diagnosis (detects changes 1-2 days after symptom onset) and assessing soft tissue involvement.
    *   Can show "striated pattern" or "checkerboard pattern" indicative of early HO.
    *   MRI with Zero Echo Time (ZTE) and TRICKS sequences can evaluate vessels and nerves, especially useful for hip NHO.
    *   Higher cost and lower specificity than bone scan for early diagnosis.

**Proposed Screening Protocol for Hungarian Inpatient Neurorehabilitation Centers:**

1.  **Risk Stratification on Admission:** Identify high-risk patients based on neurological injury severity, polytrauma, prolonged immobilization, spasticity, and other risk factors (Section 2.3).
2.  **Routine Clinical Monitoring:** Incorporate regular joint assessments (ROM, pain, swelling, warmth) into daily inpatient care protocols, especially for high-risk patients.
3.  **Clinical Suspicion Trigger:** If clinical signs and symptoms of NHO are present:
    *   **Initial Imaging:** Order bedside ultrasound as the first-line imaging modality.
    *   **Biomarker Assessment:** Consider measuring serum ALP and CRP levels.
4.  **Confirmatory Imaging (if US is positive or inconclusive and clinical suspicion persists):**
    *   Order three-phase bone scan for early confirmation of HO activity.
    *   Order plain radiographs to document baseline and monitor progression (especially after 3-6 weeks).
    *   Consider CT or MRI, particularly for pre-surgical planning and detailed anatomical assessment.
5.  **Documentation and Follow-up:** Document all findings, monitor progression, and adjust management plan accordingly.

**4. Recommended Medical Management Strategies**

**4.1. Prophylaxis (High-Risk Patients, Early Post-Injury, within 3 weeks):**

*   **Nonsteroidal Anti-inflammatory Drugs (NSAIDs):**
    *   **Indomethacin:** Gold standard for prophylaxis. Dose: 25mg standard release three times daily or 75mg long-acting daily for 3-6 weeks. Consider for at least 15 days.
    *   **Rationale:** Inhibits cyclooxygenase (COX), reducing prostaglandin E2 production, which is crucial for osteoblast differentiation.
    *   **Evidence:** Level 1 evidence for efficacy in preventing HO post-SCI.
    *   **Considerations:**
        *   GI side effects (gastritis, bleeding), renal dysfunction, potential to delay fracture healing, especially in polytrauma patients. Consider gastroprotection.
        *   Contraindicated in patients with active peptic ulcer disease, bleeding disorders, renal insufficiency, and Cerebral Amyloid Angiopathy (CAA).
        *   Monitor renal function and GI symptoms.
        *   Use with caution in patients with TBI as some data suggest potential to worsen outcomes.
        *   Celecoxib (selective COX-2 inhibitor) may be an alternative with fewer GI side effects, but less evidence in neurogenic HO.
*   **Bisphosphonates (Etidronate Disodium):**
    *   **Dose:** 20mg/kg/day for two weeks followed by 10mg/kg/day for ten additional weeks or 300mg twice daily for 6 months (various regimens reported).
    *   **Rationale:** Inhibits hydroxyapatite crystal formation and mineralization of osteoid, delaying mineralization of osteoid and reducing bone resorption.
    *   **Evidence:** May decrease HO development if initiated early (within a week of injury and before radiographic evidence), Level 2 evidence for efficacy in TBI.
    *   **Considerations:**
        *   Less effective once bone mineralization has occurred.
        *   Potential side effects: increased serum phosphate, abdominal distress, nausea, vomiting, diarrhea, may delay fracture healing, osteomalacia with long-term use.
        *   Monitor serum phosphate levels.
*   **Pulse Low-Intensity Electromagnetic Field Therapy (PLIMF):**
    *   **Therapy Characteristics:** Induction of 10 mT, frequency of 25 Hz and duration of 30 min, given for 4 weeks.
    *   **Rationale:** May increase blood flow and oxygenation, potentially removing neurogenic stimulus for HO formation.
    *   **Evidence:** Level 1 evidence from a single RCT supports prophylactic use in SCI.
    *   **Considerations:** Further research needed to fully understand role, availability in Hungary may be limited.
*   **Warfarin (Observational, low evidence):**
    *   Considered in patients requiring anticoagulation for other indications (e.g., DVT prophylaxis).
    *   **Rationale:** Vitamin K inhibitor, may inhibit bone matrix formation.
    *   **Evidence:** Level 5 evidence (observational study) suggests potential preventive association.
    *   **Considerations:** Hemorrhage risk outweighs potential HO prevention benefit as primary prophylaxis.

**4.2. Treatment of Established NHO (Once Clinically or Radiographically Confirmed):**

*   **Nonsteroidal Anti-inflammatory Drugs (NSAIDs):**
    *   **Indomethacin:** Can be used for analgesia and to reduce inflammation in established HO.
    *   **Rationale:** Symptomatic relief, anti-inflammatory action.
    *   **Evidence:** Limited evidence to reverse established HO, primarily for symptom control.
    *   **Considerations:** As above for prophylaxis.
*   **Bisphosphonates (Etidronate Disodium, Pamidronate):**
    *   **Etidronate:** May halt progression of HO, especially if initiated early (within 3-6 weeks of symptom onset).
    *   **Pamidronate:** Newer nitrogen-containing bisphosphonates may be more potent than etidronate, but limited data in neurogenic HO.
    *   **Rationale:** Inhibit bone resorption and mineralization, may slow down HO progression.
    *   **Evidence:** Level 2 evidence for etidronate to halt progression of HO.
    *   **Considerations:** Less effective once mineralization is advanced.
*   **Radiation Therapy (RT):**
    *   **Dose:** 20 Gy in 2 Gy fractions (example protocol). Single dose radiation also reported effective.
    *   **Rationale:** Inactivates pluripotent osteoprogenitor cells, prevents differentiation of mesenchymal progenitor cells into osteoblasts.
    *   **Evidence:** Effective in halting progression of early-stage NHO, may reduce pain and improve ROM in established HO.
    *   **Considerations:**
        *   More costly than NSAIDs.
        *   Potential side effects: delayed wound healing, potential to delay bone fracture healing, long-term side effects not well understood, contradictory findings in some studies (increased HO).
        *   Consider for patients with established HO not responding to conservative management, or as secondary prophylaxis after surgical excision.
*   **Corticosteroids (Limited Role):**
    *   Not recommended for routine NHO management.
    *   Considered for short-term use to reduce inflammation and edema in specific situations, similar to their use in FOP flare-ups, but not for chronic treatment due to side effects.

**5. Recommended Rehabilitation Interventions**

*   **Early and Gentle Range of Motion (ROM) Exercises:**
    *   Crucial for maintaining joint mobility and preventing contractures.
    *   Perform active-assisted or passive ROM exercises within pain-free limits.
    *   Avoid forceful or aggressive passive movements, especially in early stages or during acute inflammation, as it can exacerbate HO and cause microtrauma.
    *   Gentle mobilization can be initiated 1 week after surgical excision.
*   **Spasticity Management:**
    *   Aggressive and proactive management of spasticity is essential, as spasticity is a major risk factor for HO.
    *   Utilize a multimodal approach:
        *   Pharmacological: Oral medications (baclofen, tizanidine), botulinum toxin injections, intrathecal baclofen.
        *   Non-Pharmacological: Physical therapy, stretching, positioning, serial casting.
*   **Positioning and Splinting:**
    *   Implement meticulous positioning strategies to prevent contractures and maintain optimal joint alignment.
    *   Use splints (e.g., night splints in elbow extension) to preserve range of motion, especially after surgical excision.
*   **Pain Management:**
    *   Address pain effectively using pharmacological and non-pharmacological approaches.
    *   Pain management is crucial to facilitate participation in rehabilitation and improve quality of life.
*   **Functional Training:**
    *   Adapt functional training programs to accommodate ROM limitations due to NHO.
    *   Focus on maximizing independence in ADLs and mobility within the patient's functional capacity.

**6. Surgical Guidelines and Indications**

**6.1. Indications for Surgical Excision:**

*   **Functional Limitation:** Significant functional impairment due to HO, limiting mobility, transfers, hygiene, and ADLs.
*   **Pain:** Intractable pain that cannot be alleviated by conservative measures.
*   **Neurovascular Compression:** Compression of nerves or blood vessels causing pain, neurological deficits, or vascular compromise.
*   **Pressure Ulcers:** Amelioration of bedsores caused or exacerbated by HO.
*   **Joint Ankylosis:** Complete or near-complete joint ankylosis due to HO.

**6.2. Timing of Surgical Excision:**

*   **Maturity of HO:** Ideally, surgery should be delayed until HO has matured and inflammation has subsided, typically 6-18 months after HO initiation.
*   **Clinical Necessity:** In some cases, earlier surgical intervention may be necessary if HO is causing severe functional limitations or neurovascular compression, even if not fully mature.
*   **Assessment of Maturity:**
    *   Clinical judgment: Reduced pain and inflammation.
    *   Radiographs: Stable neocortex on serial radiographs.
    *   Bone Scan: Serial Technetium 99 bone scans to assess for decreased activity.
    *   Serum Alkaline Phosphatase (ALP): Baseline values of ALP should be considered preoperatively.

**6.3. Surgical Technique and Pre-operative Planning:**

*   **Multidisciplinary Decision:** Surgical decision should be made collegially, involving the patient, PM&R physician, orthopedic surgeon, and other relevant specialists. Functional contract specifying expected functional gain should be formulated.
*   **Preoperative Imaging:** Contrast-enhanced 3D CT angiography is essential for surgical planning to visualize the extent of HO and its relationship to neurovascular structures. Consider MRI with ZTE and TRICKS sequences for hip NHO.
*   **Surgical Approach:** Direct approach towards the lesion, utilizing existing incisions if possible. Functional resection is preferable, removing only the part of HO that limits joint motion, respecting implantation bases to avoid iatrogenic fracture.
*   **Embolization:** Consider preoperative embolization of the feeding artery to reduce intraoperative bleeding risk.

**6.4. Postoperative Management:**

*   **Prophylaxis against Recurrence:**
    *   NSAIDs (Indomethacin): Postoperative administration for at least 1 week, up to 6 weeks.
    *   Radiotherapy: Consider single-dose or fractionated radiotherapy postoperatively, especially in high-risk patients or medullary lesion, although may increase infection risk in medullary lesion.
    *   Bisphosphonates: Consider postoperative bisphosphonate treatment, although evidence is limited.
*   **Rehabilitation:**
    *   Early and aggressive postoperative rehabilitation is crucial.
    *   Continuous Passive Motion (CPM) machines may be beneficial.
    *   Gentle range of motion exercises should be initiated within pain-free limits, starting approximately 1 week post-surgery.
*   **Infection Control:** Strict adherence to infection control protocols is paramount due to increased infection risk, especially with combined surgery and radiation.

**7. Prognostic Factors, Functional Outcomes, and Long-Term Follow-Up**

**7.1. Prognostic Factors:**

*   **Severity of Initial Injury:** More severe neurological injuries (TBI, SCI) are associated with higher risk and potentially poorer outcomes.
*   **Polytrauma:** Presence of polytrauma increases risk and complexity of management.
*   **Spasticity:** Severe spasticity is a strong risk factor for both HO development and recurrence.
*   **Timing of Intervention:** Early diagnosis and intervention (prophylaxis and treatment) are associated with better outcomes. Delayed surgical resection may lead to less functional improvement.
*   **Completeness of Resection:** Incomplete surgical resection increases recurrence risk.
*   **Cognitive Impairment:** Severe cognitive impairment may be associated with poorer functional outcomes and increased recurrence risk.

**7.2. Functional Outcomes:**

*   **Primary Goal:** Improve range of motion, reduce pain, enhance functional mobility, and improve quality of life.
*   **Expected Gains:** Surgical excision can effectively increase ROM, improve function, and reduce pain.
*   **Potential Limitations:** Functional improvement may be limited in patients with severe neurological deficits or significant pre-existing functional impairments.

**7.3. Long-Term Follow-Up:**

*   **Recurrence Monitoring:** Regular clinical assessments and imaging (radiographs) to monitor for HO recurrence, especially after surgical excision. Recurrence rates reported up to 30% or higher.
*   **Functional Assessment:** Periodic functional assessments to track progress and address any ongoing functional limitations.
*   **Biomarker Monitoring (Optional):** Consider periodic serum ALP level monitoring.
*   **Duration:** Long-term follow-up is recommended for at least 12-18 months post-injury, and potentially longer, given the risk of recurrence and long-term functional implications.

**8. Clinical Algorithms and Step-by-Step Approaches**

**(Proposed Clinical Algorithm for NHO Management in Hungarian Inpatient Neurorehabilitation Centers)**

**Step 1: Risk Stratification and Early Recognition (Admission and Throughout Inpatient Stay)**

*   Identify high-risk patients (TBI/SCI with risk factors - Section 2.3).
*   Implement routine clinical monitoring for early signs of NHO (Section 3.1).

**Step 2: Diagnostic Workup (If Clinical Suspicion Arises)**

*   **Initial Imaging:** Bedside Ultrasound (US).
*   **Biomarkers:** Serum ALP and CRP levels.
*   **Confirmatory Imaging (If US positive/inconclusive):** Three-phase bone scan, plain radiographs, CT/MRI as needed.

**Step 3: Medical Management**

*   **Prophylaxis (High-Risk Patients, Early Post-Injury):** NSAIDs (Indomethacin) +/- PLIMF (if available).
*   **Treatment (Established NHO):** NSAIDs (Indomethacin) for symptom control, consider Bisphosphonates (Etidronate) in early stages, consider Radiation Therapy for established HO or post-surgical recurrence prevention.

**Step 4: Rehabilitation Interventions (Throughout Inpatient Stay)**

*   Gentle ROM exercises within pain-free limits.
*   Aggressive Spasticity Management.
*   Positioning and Splinting.
*   Pain Management.
*   Functional Training.

**Step 5: Surgical Referral and Management (If Conservative Measures Fail)**

*   Refer to orthopedic surgeon experienced in HO excision.
*   Pre-operative planning with CT angiography/MRI.
*   Surgical excision after HO maturation.
*   Post-operative prophylaxis (NSAIDs, +/- Radiotherapy).
*   Aggressive Post-operative Rehabilitation.

**Step 6: Long-Term Follow-Up (Outpatient Setting)**

*   Regular clinical assessments and imaging for recurrence monitoring.
*   Ongoing rehabilitation to maximize function.

**Step 7: Patient and Family Education (Throughout Care)**

*   Educate about NHO risk factors, signs, management, and importance of adherence to treatment plan.

**9. Novel or Emerging Therapies**

*   **Targeting OSM Signaling:** Inhibition of Oncostatin M (OSM) signaling.
*   **Substance P (SP) Antagonism:** Targeting Substance P and its receptor NK1r.
*   **RAR-γ Agonists (Palovarotene):** Retinoic acid receptor-gamma agonists show promise in preventing HO progression.
*   **HIF-1α Inhibition:** Pharmacological inhibition of HIF-1α.
*   **BMP Inhibitors:** Noggin, BMP receptor inhibitors.
*   **Free Radical Scavengers:** Allopurinol, N-acetylcysteine.
*   **Imatinib (PDGF inhibitor):** Imatinib also blocks HIF1a and affects HO progression.
*   **MicroRNAs (miRNA) modulation:** Targeting osteo-suppressive miRNAs.
*   **Raman Spectroscopy:** Emerging diagnostic tool for early detection.

These novel therapies are primarily in preclinical or early clinical trial stages and are not yet standard clinical practice, but represent potential future directions for NHO management. PM&R specialists in Hungary should stay updated on emerging research in this area.

**10. Practical Inpatient Care Considerations for PM&R Specialists in Hungary**

*   **Resource-Aware Approach:** Utilize bedside ultrasound as a cost-effective and readily available early diagnostic tool. Prioritize plain radiography for initial imaging. Reserve bone scans and advanced imaging (CT/MRI) for specific indications.
*   **Medication Availability and Cost-Effectiveness:** Consider the availability and cost of NSAIDs (Indomethacin, Celecoxib), Bisphosphonates (Etidronate, Pamidronate), and emerging therapies within the Hungarian healthcare system and reimbursement framework. Prioritize cost-effective NSAID prophylaxis for high-risk patients.
*   **Multidisciplinary Collaboration:** Establish strong collaboration with orthopedic surgeons, radiologists, and rehabilitation therapists to ensure seamless patient care.
*   **Patient Education in Hungarian Language:** Develop patient education materials in Hungarian language to facilitate patient and family understanding and adherence to treatment plans.
*   **Focus on Prevention and Early Intervention:** Emphasize early recognition, risk factor management (spasticity, pressure ulcers), and prophylactic strategies (NSAIDs, gentle ROM) to minimize NHO development and its impact on functional outcomes.
*   **Tailored Rehabilitation Programs:** Design individualized rehabilitation programs that address the specific functional limitations imposed by NHO, focusing on gentle ROM, spasticity management, and functional training within pain-free limits.
*   **Access to Radiotherapy and Surgical Expertise:** Understand referral pathways for radiotherapy and surgical consultation within the Hungarian healthcare system.

This comprehensive guideline is intended to provide PM&R specialists in Hungarian neurorehabilitation centers with a robust framework for managing NHO, incorporating the latest evidence and practical considerations for inpatient care. Continuous professional development and staying updated with emerging research are crucial for optimizing patient outcomes in this challenging condition.